# STRIPS Action Model

En este notebook se implementa un planificador basado en _STRIPS_ que permita a un mono situado en una posición cualquiera alcanzar unos plátanos colgados del techo mediante el uso de una caja que podrá desplazar por el suelo para subirse a ella y coger los plátanos.

Las reglas son:

- Para alcanzar los plátanos el mono debe subirse a la caja cuando la caja y éste estén en la misma posición que los plátanos.
- El mono puede empujar la caja cuando se encuentra en el suelo y en la misma posición que la caja.
- El mono no puede desplazarse horizontalmente cuando está subido sobre la caja.

![](./STRIPS.svg)

Las clases y métodos usados en el notebook para resolver la planificación pueden encontrarse en el paquete de Python ([`cdalvaro`](cdalvaro)) anexo a este documento.

Es paquete modeliza las propiedades ([`properties`](cdalvaro/properties)) de los estados ([`State`](cdalvaro/state.py)) y las acciones ([`actions`](cdalvaro/actions)) que se pueden realizar con la lógica y restricciones del problema.

Por otro lado, las clases [`Heuristic`](cdalvaro/heuristic.py) y [`Strips`](cdalvaro/strips.py) contienen la lógica para la búsqueda de la planificación que permita al mono alcanzar el plátano.

### Tabla de Contenidos

- [Condición inicial](#Condición-inicial)
- [Generación de la planificación](#Generación-de-la-planificación)
- [Plan encontrado](#Plan-encontrado)
- [Algoritmo STRIPS](#Algoritmo-STRIPS)
- [Heurística del problema](#Heurística-del-problema)
- [Pesos de acciones y propiedades](#Pesos-de-acciones-y-propiedades)
  - [Pesos de acciones](#Pesos-de-acciones)
  - [Pesos de propiedades](#Pesos-de-propiedades)
- [Acciones](#Acciones)
- [Propiedades](#Propiedades)
- [Conclusiones](#Conclusiones)
  - [Condición inicial alternativa 1](#Condición-inicial-alternativa-1)
  - [Condición inicial alternativa 2](#Condición-inicial-alternativa-2)
  - [Condición inicial alternativa 3](#Condición-inicial-alternativa-3)

### Condición inicial

Comenzamos por definir la condición inicial del problema y el objetivo a alcanzar:

In [8]:
from cdalvaro.element import Banana, Box, Monkey
from cdalvaro.properties import GroundLevel, TopLevel, AtLevel, AtPosition, Has
from cdalvaro.state import State

initial_state = State({
    AtPosition(Monkey(), 1),
    AtLevel(Monkey(), GroundLevel()),
    AtPosition(Banana(), 2),
    AtPosition(Box(), 3)
})

goal_state = State({
    Has(Monkey(), Banana())
})

En esta primera parte se definen las condiciones iniciales del problema, donde se establecen la posición y nivel del mono, y las posiciones de la caja y del plátano. No es necesario establecer los niveles de los dos últimos, dado que están fijados por las restricciones del problema. Los plátanos siempre están en el nivel superior, y la caja siempre está en el nivel inferior.

Así, las condiciones de inicio del problema son:

- El mono está situado en la posición 1: `AtPosition(Monkey(), 1)`
- El mono está en el nivel inferior: `AtLevel(Monkey(), GroundLevel())`
- El plátano está en la posición 2: `AtPosition(Banana(), 2)`
- La caja está en la posición 3: `AtPosition(Box(), 3)`

Para el estado objetivo, cabe destacar la simplicidad del estado: `Has(Monkey(), Banana())`. _El mono tiene el plátano_.

No se requiere definir más propiedades del estado final, dado que ya vienen determinadas por la inmutabilidad de la propiedad de posición del plátano en la condición inicial.

### Generación de la planificación

Con el estado inicial y final ya definidos, se crean una instancia de `Heuristic` y otra de `Strips` que conjuntamente generarán el plan para que el mono alcance el plátano.

In [9]:
from cdalvaro.actions import BaseAction
from cdalvaro.heuristic import Heuristic
from cdalvaro.strips import Strips

BaseAction.verbose = False

planner = Strips(initial_state, goal_state, Heuristic(initial_state))

La clase [`Heuristic`](cdalvaro/heuristic.py) implementa los métodos:

- `possible_actions(state: State) -> Actions`: genera todas las posibles acciones disponibles en el escenario, y tiene en cuenta el estado para construir la acción de conseguir el plátano. (Usa `state` para determinar si en ese estado es posible la acción de conseguir el plátano).
- `choose_actions(self, state: State, goal: State) -> Actions`: devuelve ordenadas de más efectiva a menos efectiva las acciones que llevan del estado `state` al estado `goal`.
- `sort_properties(properties: Properties) -> List[BaseProperty]`: ordena las propiedades indicadas en base al peso de cada propiedad.

(En la sección [_Pesos de acciones y propiedades_](#Pesos-de-acciones-y-propiedades) se detalla en qué consisten estos pesos y el criterio de asignación.)

La clase [`Strips`](cdalvaro/strips.py) por su parte, implementa el método: `get_plan(self) -> Union[Actions, bool]`, que devuelve la planificación encontrada con la lista de acciones a realizar para que el mono consiga el plátano, o `False` si no se consigue encontrar una planificación.

Este método parte del estado objetivo determinando las acciones en sentido inverso que habría que aplicar para llegar al estado inicial. (El plan devuelto por el algoritmo sí está en el orden de aplicación correcto para aplicarlo sobre el estado inicial).

En el apartado [Algoritmo STRIPS](#Algoritmo-STRIPS) se describe el código que calcula la planificación.

### Plan encontrado

In [10]:
def get_plan(planner: Strips, optimal_solution: bool = True):
    """
    Método auxiliar para mejorar la legibilidad del documento.
    Muestra la planificación obtenida usando un planificador basado en STRIPS.
    
    Args:
        planner (Strips): Planificador usado para calcular la planificación.
        optimal_solution (bool, optional): Flag para activar/desactivar la búsqueda del plan óptimo. Default: True.
    """
    Strips.efficiency_limit = 10 if optimal_solution else 0
    plan = planner.get_plan()
    if not plan:
        print("No se ha conseguido eleborar un plan que resuelva el problema ❌")
    else:
        print("Plan encontrado ✅")
        for it, action in zip(range(1, len(plan) + 1), plan):
            print(f"  {it}. {action}")

get_plan(planner)

Plan encontrado ✅
  1. Mueve 🐒 de 1 a 3
  2. 🐒 empuja la 📦 de 3 a 2
  3. Cambia 🐒 al nivel Superior en posición 2
  4. 🐒 consigue 🍌 en posición 2


### Algoritmo STRIPS

A continuación se describe el algoritmo implementado en el método [`get_plan()`](cdalvaro/strips.py) de la clase `Strips`. (Se recomienda leer esta descripción con el código en paralelo).

- El algoritmo obtiene una lista de objetivos (`targets`) compuesta por las propiedades del estado `goal` e inicializa su estado _actual_ (`state`) al estado de partida del problema.

- A continuación comienza a iterar sobre `targets` hasta que no queden más en la lista.

- En cada iteración se extrae el primer elemento de la lista (`target`) y se comprueba si se trata de una acción ([`BaseAction`](cdalvaro/actions/base_action.py)) o si por el contrario es una propiedad ([`BaseProperty`](cdalvaro/properties/base_property.py)).

- Si `target` es una acción, se comprueba si ésta puede generar un estado válido a partir del estado actual `state`. Si es así, se actualiza `state` al nuevo estado y la acción se añade al plan.

- Si `target` es una propiedad, se comprueba si esta propiedad pertenece a las propiedades del estado actual. Si se cumple la condición, se ignora la propiedad y se continúa explorando el resto de objetivos de la lista.

- Si `target` es una propiedad y no se encuentra entre las propiedades del estado que se está analizando, se procede a buscar las acciones que pueden dar lugar desde el estado actual `state` a un estado que contiene la propiedad `target` que se está evaluando.

- Esta lista de posibles acciones la genera la heurística con el método [`choose_actions`](cdalvaro/heuristic.py) que está detallado en la sección [Heurística del problema](#Heurística-del-problema).

- Las acciones devueltas por la heurística vienen ordenadas de mayor a menor peso. Es decir, de más a menos óptima.

- Se comprueba si la lista de acciones devuelta por la heurística está vacía. De ser así, no es posible alcanzar el estado objetivo y no se consigue encontrar la planificación buscada, por lo que se acaba devolviendo `False`.

- Si la lista de acciones no está vacía, se selecciona la primera acción de la lista (por ser la más eficiente) y se inserta como primer elemento de la lista de `targets`. Además se insertan en la lista de objetivos, también por delante, las precondiciones de la acción seleccionada para buscar las acciones que puedan llevar a aplicar posteriormente la acción encontrada.

- Por último, se implementa un mecanismo de seguridad para evitar caer en búcles infinitos. Si pasados `10` ciclos de búsqueda de acciones no se ha llegado al final del plan, se elimina el orden de acciones devuelto por la heurística desordenando aleatoriamente la lista de acciones obtenida. De este modo, aunque el plan tarde más en converger y no sea óptimo acabará encontrando una combinanción de acciones que resuelva el plan.

### Heurística del problema

La heurísitica que se encarga de identificar las acciones posibles y de determinar cuál es la más eficiente se encuentra en la clase [`Heuristic`](cdalvaro/heuristic.py).

Esta clase está compuesta por los métodos:

- `possible_actions(state: State) -> Actions`: Encargado de generar todas las posibles acciones del problema, incluída la acción de conseguir el plátano. Recibe el parámetro `state` para determinar bajo qué posición se puede generar la acción de conseguir el plátano.

- `choose_actions(self, state: State, goal: State) -> Actions`: Se encarga de determinar las posibles acciones que se pueden aplicar a un estado de partida `state` para generar un estado objetivo `goal`.

La determinación de si una acción es aplicable o no al estado `state` para generar el estado `goal` consiste en calcular la intersección entre las propiedades de la _lista de añadir_ de la acción y las propiedades del estado `goal`. Si la intersección no es vacía entonces se considera que la acción es válida.

Una vez determinadas las acciones aplicables, se ordena de mayor a menor nivel de eficiencia.

El criterio seguido para determinar el nivel de eficiencia es el siguiente:

1. Se consideran más eficientes las acciones donde la suma de los pesos de las propiedades de la intersección resultante entre las propeidades del pseudo-estado que genera el estado `goal` a partir de aplicar la acción analizada y las propiedades del estado de partida del problema es mayor.
2. En caso de que haya acciones donde la primera condición tenga el mismo valor, se tiene en cuenta primero aquellas acciones con mayor peso intrínseco.

(Esta explicación queda más clara viendo la implementación de los métodos `Heuristic.choose_actions` y `Heuristic._properties_weight` donde el primero determina y ordena en base al segundo método, las posibles acciones a realizar dado un estado de partida y un estado objetivo).

### Pesos de acciones y propiedades

Para ayudar a la heurística a determinar qué acciones son preferibles frente a otras, y de igual manera, qué propiedades son prioritarias, se han asignado unos pesos a cada tipo de acción y propiedad.

#### Pesos de acciones

Para resolver el problema se han creado cuatro tipos de acciones diferentes:

- [`GetBanana`](cdalvaro/actions/get_banana.py) - Peso `4`: Se considera esta acción como la prioritaria frente al resto. Lo que hace que cuando el mono esté sobre la caja, la acción de conseguir el plátano sea preferente a bajarse de nuevo de la caja.

- [`ChangeLevel`](cdalvaro/actions/change_level.py) - Peso `3`: Este peso hace que la acción de subirse a la caja sea preferente a que el mono vuelva a empujar la caja o se mueva por el suelo.

- [`MoveHorizontally`](cdalvaro/actions/move_horizontally.py) - Peso `2`: Este peso hace que la acción sea prioritaria frente a la acción de mover la caja, lo que permite al planificador que una vez el mono ha alcanzado la caja no siga empujándola indefinidamente. Esto es así, porque el peso de las acciones se tiene en cuenta por detrás del peso de las propiedades, por lo que sólo sirve para evitar que el programa entre en bucles infinitos con el mono empujando incansablemente la caja.

- [`PushBox`](cdalvaro/actions/push_box.py) - Peso `1`: La acción de mover la caja es la que menor peso tiene frente al resto. De nuevo porque los pesos de las propiedades son preferentes. Esto hace que finalmente se elijan las acciones de mover la caja frente a las de movimiento. En definitiva, asignar el menor valor a la acción evita que el programa entre en bucles infinitos en la planificación.

**Nota**: Hay que tener en cuenta que el valor absoluto de los pesos no importa, sólo el valor relativo entre ellos para dar prioridad a unas acciones frente a otras.

#### Pesos de propiedades

Igual que con las acciones, existen tres propiedades de estados que modelizan el problema:

- [`Has`](cdalvaro/properties/has.py) - Peso `0`: Esta propiedad no se prioriza frente a ninguna. Representa la propiedad de que un objeto posea otro, por ejemplo, que el mono tenga el plátano.

- [`AtLevel`](cdalvaro/properties/at_level.py) - Peso `0`: Esta propiedad tampoco se prioriza frente al resto. Simboliza la propiedad de que el mono esté en un nivel concreto.

- [`AtPosition`](cdalvaro/properties/at_position.py) - Peso `?`: El peso de esta propiedad depende del elemento que tenga asignado:
  
  - Si el elemento de la propiedad es [`Banana()`](cdalvaro/element.py), el peso es `1`.
  - Si el elemento de la propiedad es [`Monkey()`](cdalvaro/element.py), el peso es `2`.
  - Si el elemento de la propiedad es [`Box()`](cdalvaro/element.py), el peso es `3`.

De aquí se ve que el mayor peso corresponde a la posición de la caja, lo que hace que el mono intente desplazarse a la posición de la caja lo antes posible.

### Acciones

Todas las acciones heredan de la clase [`BaseAction`](cdalvaro/actions/base_action.py). Que contiene un nombre `name` que identifica la acción y un peso `weight`.

Cada clase hija de la clase base debe implementar los métodos: `_set_precondition`, `_set_add_list` y `_set_remove_list` que definirán las propiedades de cada una de las listas de _precondición, añadir y eliminar_ de la acción representada respectivamente.

Además la clase `BaseAction` implementa los métodos:

- `can_apply(self, state: State, reverse: bool) -> bool`: que permite determinar si se puede aplicar una acción sobre un estado dado. O saber si podría obtenerse un estado válido a partir de revertir la acción sobre el estado `state`.

- `apply(self, state: State, reverse: bool = False) -> Union[State, None]`: para obtener el estado resultante de aplicar la acción sobre el estado `state`, o para obtener el _pseudo-estado_ que generaría el estado `state` a partir de revertir la acción.

### Propiedades

La clase [`BaseProperty`](cdalvaro/properties/base_property.py) modeliza una propiedad de estado.

Como se ha visto ya, las clases hijas se usarán para definir las propiedades de los estados y de las acciones del problema.

### Conclusiones

Como ha podido verse en la sección [Plan encontrado](#Plan-encontrado), la solución encontrada al problema es óptima. El mono se desplaza en un sólo movimiento a la posición de la caja, la empuja a la posición del plátano en un solo movimiento, trepa y consigue el plátano. _Cuatro movimientos en total._

Puede verse, que si se desactiva la heurística _eficiente_ estableciendo `Strips.efficiency_limit = 0` el mono puede encontrar la solución óptima o no. _Pero, al final acaba encontrando una solución viable._

In [11]:
get_plan(planner, optimal_solution=False)

AVISO: La heurística no está encontrando soluciones eficientes
Plan encontrado ✅
  1. Mueve 🐒 de 1 a 2
  2. Mueve 🐒 de 2 a 3
  3. 🐒 empuja la 📦 de 3 a 2
  4. 🐒 empuja la 📦 de 2 a 3
  5. 🐒 empuja la 📦 de 3 a 2
  6. 🐒 empuja la 📦 de 2 a 3
  7. Mueve 🐒 de 3 a 1
  8. Mueve 🐒 de 1 a 2
  9. Mueve 🐒 de 2 a 3
  10. 🐒 empuja la 📦 de 3 a 1
  11. 🐒 empuja la 📦 de 1 a 2
  12. 🐒 empuja la 📦 de 2 a 1
  13. 🐒 empuja la 📦 de 1 a 2
  14. Cambia 🐒 al nivel Superior en posición 2
  15. 🐒 consigue 🍌 en posición 2


Por otro lado, a continuación se presentan otros ejemplos, algunos más sencillos, otros más complicados para mostrar la versatilidad del algoritmo.

#### Condición inicial alternativa 1

El mono comienza ubicado en el suelo, en la misma posición que la caja, y en distinta posición que el plátano.

En este caso, el mono sólo debería empujar la caja a la posición del plátano, subirse sobre ella y conseguir su objetivo.

In [12]:
initial_state = State({
    AtPosition(Monkey(), 1),
    AtLevel(Monkey(), GroundLevel()),
    AtPosition(Banana(), 2),
    AtPosition(Box(), 1)
})

planner = Strips(initial_state, goal_state, Heuristic(initial_state))
get_plan(planner)

Plan encontrado ✅
  1. 🐒 empuja la 📦 de 1 a 2
  2. Cambia 🐒 al nivel Superior en posición 2
  3. 🐒 consigue 🍌 en posición 2


#### Condición inicial alternativa 2

Ahora, el mono comienza subido sobre la caja en una posición distinta a la ubicación del plátano.

En la solución óptima, el mono tendrá que bajarse de la caja, empujarla a la posición del plátano, trepar por ella y conseguir el plátano.

In [13]:
initial_state = State({
    AtPosition(Monkey(), 1),
    AtLevel(Monkey(), TopLevel(), 1),
    AtPosition(Banana(), 2),
    AtPosition(Box(), 1)
})

planner = Strips(initial_state, goal_state, Heuristic(initial_state))
get_plan(planner)

Plan encontrado ✅
  1. Cambia 🐒 al nivel Inferior en posición 1
  2. 🐒 empuja la 📦 de 1 a 2
  3. Cambia 🐒 al nivel Superior en posición 2
  4. 🐒 consigue 🍌 en posición 2


#### Condición inicial alternativa 3

Por último, probamos la situación en la que el mono, la caja y el plátano están en la misma posición y el mono está en el nivel inferior.

En este caso el mono sólo deberá trepar por la caja y conseguir el plátano.

In [14]:
initial_state = State({
    AtPosition(Monkey(), 2),
    AtLevel(Monkey(), GroundLevel()),
    AtPosition(Banana(), 2),
    AtPosition(Box(), 2)
})

planner = Strips(initial_state, goal_state, Heuristic(initial_state))
get_plan(planner)

Plan encontrado ✅
  1. Cambia 🐒 al nivel Superior en posición 2
  2. 🐒 consigue 🍌 en posición 2


Como puede verse, en todos los casos, la heurística consigue encontrar la solución óptima (salvo en el caso en el que se ha desactivado explícitamente).

Es muy interesante ver, cómo este sistema de propiedades de estados donde sólo se tienen en cuenta propiedades parciales, permite generar una planifiación para alcanzar el objetivo propuesto.